# The starting point for GW/BSE: QE scf+nscf calculations.

In the following it is shown how to run quickly a QE scf+nscf flow in order to have a starting ground state density to perform GW and BSE simulations. For more details, please have a look at the AiiDA tutorials: https://aiida-tutorials.readthedocs.io/en/latest/. 

You should run these steps quickly, without trying to fully understand what is happening here. The inputs are then explained in more detail in the next parts of the tutorial.


## prerequisites: 

We suppose that you have successfully installed AiiDA, aiida-quantuespresso and you have stored computer, codes, structures and pseudo in the AiiDA database.

In [1]:
%aiida  
## this is a magic line function, it can be substitued by: ---

from aiida import orm

## 1 - Creation of the scf workflow instance

Here we start directly creating an instance of a workflow, not of a calcjob, to have error handling support and automatic input creation (protocols).

The idea is to create a builder instance, which collects all the inputs needed for the simulation, then submit it by means of the aiida "submit" function.

In [2]:
from aiida.plugins import WorkflowFactory

In [3]:
PwBaseWorkchain = WorkflowFactory('quantumespresso.pw.base') #try also quantumespresso.pw.relax...

The instance of the workchain, with automatic inputs creation, can be obtained using the .get_builder_from_protocol method of the workchain, if provided codes, structure:

In [4]:
from aiida_quantumespresso.common.types import ElectronicType

builder = PwBaseWorkchain.get_builder_from_protocol(
    
                code=orm.load_code('pw-6.8@hydralogin'),
                structure=orm.load_node(77395),           #use the structure store before
                protocol='fast',
                overrides={},
                electronic_type=ElectronicType.INSULATOR,
                pseudo_family="PseudoDojo/0.4/LDA/SR/standard/upf",
    
)

### 1.1 - Change the pseudos (see also "structure_and_pseudos" section)

In [5]:
family = orm.load_group("sg15")
#builder.<sublevels_up_to .pw>.pseudos = family.get_pseudos(structure=structure) 
builder.pw.pseudos = family.get_pseudos(structure=orm.load_node(77395)) 

In [6]:
builder.pw.parameters.get_dict()

{'CONTROL': {'calculation': 'scf',
  'forc_conv_thr': 0.001,
  'tprnfor': True,
  'tstress': True,
  'etot_conv_thr': 0.0002},
 'SYSTEM': {'nosym': False,
  'occupations': 'fixed',
  'ecutwfc': 30.0,
  'ecutrho': 240.0},
 'ELECTRONS': {'electron_maxstep': 80, 'mixing_beta': 0.4, 'conv_thr': 8e-10}}

## 1.2 - How to modify default inputs

It is possible to modify in two ways the inputs:

    (1) to use the overrides in the builder creation;
    (2) to re-define the dictionaries and the input in general

In [7]:
# (1):

overrides={
    'pw':{
        'parameters':{
            'SYSTEM': {
            'nosym': False,
            'occupations': 'fixed',
            'ecutwfc': 70.0,
            'ecutrho': 70*4,
            'force_symmorphic':True,
                      },
    },},}

builder = PwBaseWorkchain.get_builder_from_protocol(
    
                code=orm.load_code('pw-6.8@hydralogin'),
                structure=orm.load_node(42601),
                protocol='fast',
                overrides=overrides,
                electronic_type=ElectronicType.INSULATOR,
                pseudo_family="PseudoDojo/0.4/LDA/SR/standard/upf",
    
)

builder.pw.parameters.get_dict()

{'CONTROL': {'calculation': 'scf',
  'forc_conv_thr': 0.001,
  'tprnfor': True,
  'tstress': True,
  'etot_conv_thr': 0.0002},
 'SYSTEM': {'nosym': False,
  'occupations': 'fixed',
  'ecutwfc': 70.0,
  'ecutrho': 280,
  'force_symmorphic': True},
 'ELECTRONS': {'electron_maxstep': 80, 'mixing_beta': 0.4, 'conv_thr': 8e-10}}

In [9]:
# (2)

pw_parameters = {
  'CONTROL': {'calculation': 'scf',
  'forc_conv_thr': 0.002,
  'tprnfor': True,
  'tstress': True,
  'etot_conv_thr': 0.0002},
 'SYSTEM': {'nosym': False,
  'occupations': 'fixed',
  'ecutwfc': 70.0,
  'ecutrho': 280,
  'force_symmorphic':True,},
 'ELECTRONS': {'electron_maxstep': 90, 'mixing_beta': 0.4, 'conv_thr': 8e-10},
}

builder.pw.parameters = orm.Dict(dict=pw_parameters)

builder.pw.parameters.get_dict()

{'CONTROL': {'calculation': 'scf',
  'forc_conv_thr': 0.002,
  'tprnfor': True,
  'tstress': True,
  'etot_conv_thr': 0.0002},
 'SYSTEM': {'nosym': False,
  'occupations': 'fixed',
  'ecutwfc': 70.0,
  'ecutrho': 280,
  'force_symmorphic': True},
 'ELECTRONS': {'electron_maxstep': 90, 'mixing_beta': 0.4, 'conv_thr': 8e-10}}

Let's see the kpoints mesh that is automatically choosen by means of the kpoints_distance (inverse of the density):

In [10]:
builder.kpoints_distance.value

0.5

We can redefine the kpoints, by creating an istance of KpointsData, then setting the mesh, and eventually redefine the builder.kpoints attribute:

In [11]:
kpoints = orm.KpointsData()
kpoints.set_kpoints_mesh([4,4,4])

builder.kpoints = kpoints

Very important is also to set the workchain to not delete the remote folder, as we need it to perform nscf on top:

In [12]:
builder.clean_workdir = orm.Bool(False)

## 1.3 - Resources 
Then you should set up the options about resources and submission settings. 
These have to be stored as a python dictionary in the metadata method:

In [13]:
builder.pw.metadata.options = {
    'max_wallclock_seconds': 60*60, # in seconds
    'resources': {
            "num_machines": 1, # nodes
            "num_mpiprocs_per_machine": 16, # MPI per nodes
            "num_cores_per_mpiproc": 1, # OPENMP
        },
    'prepend_text': u"export OMP_NUM_THREADS="+str(1), # if needed
    #'account':'project_name',
    'queue_name':'s3par',
    #'qos':'',
}

## 2 - RUN!

In [14]:
from aiida.engine import submit

In [15]:
run_scf = submit(builder)

In [16]:
run_scf.pk

86340

You can control the calculation from a shell typing:
    
    verdi process list
    verdi process show <pk of the run_scf>
    verdi process report <pk of the run_scf>

In [21]:
run_scf.is_finished_ok

True

In [ ]:
run_scf.outputs.   #use tab to see available outputs

## 3 - Run the nscf workchain

In [22]:
overrides={
    'pw':{
        'parameters':{
            'CONTROL': {'calculation': 'nscf',},
            'SYSTEM': {
            'nosym': False,
            'occupations': 'fixed',
            'ecutwfc': 70.0,
            'ecutrho': 70*4,
            'nbnd':200,
            'force_symmorphic':True,
                      },
    },},}

builder = PwBaseWorkchain.get_builder_from_protocol(
    
                code=orm.load_code('pw-6.8@hydralogin'),
                structure=orm.load_node(42601),
                protocol='fast',
                overrides=overrides,
                electronic_type=ElectronicType.INSULATOR,
                pseudo_family="PseudoDojo/0.4/LDA/SR/standard/upf",
    
)

#change the pseudos to be consistent with the scf.
family = orm.load_group("sg15")
#builder.<sublevels_up_to .pw>.pseudos = family.get_pseudos(structure=structure) 
builder.pw.pseudos = family.get_pseudos(structure=orm.load_node(42601)) 

builder.pw.metadata.options = {
    'max_wallclock_seconds': 60*60, # in seconds
    'resources': {
            "num_machines": 1, # nodes
            "num_mpiprocs_per_machine": 16, # MPI per nodes
            "num_cores_per_mpiproc": 1, # OPENMP
        },
    'prepend_text': u"export OMP_NUM_THREADS="+str(1), # if needed
    #'account':'project_name',
    'queue_name':'s3par',
    #'qos':'',
}

builder.clean_workdir = orm.Bool(False)

## Setting the parent scf calculation as starting point for charge density

We set as parent_folder the previous completed scf run. To understand if run_scf is completed and went ok:

    run_scf.is_finished_ok

In [23]:
builder.pw.parent_folder = run_scf.outputs.remote_folder

In [24]:
run_nscf = submit(builder)

In [25]:
run_nscf.pk

86354

In [27]:
run_nscf.is_finished_ok

True